In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

In [33]:
course

,id,date_start,close_date,knowledge_area_id
0,3,2019-03-15 00:00:00,2019-06-15 20:59:00,1
1,35,2019-11-01 00:00:00,2020-02-29 20:59:00,1
2,89,2020-03-18 00:00:00,2020-08-31 20:59:00,1
3,237,2020-09-04 00:00:00,2020-11-30 20:59:00,1


In [35]:
pd.to_datetime(course.date_start) - pd.to_datetime(course.close_date)

0    -93 days +03:01:00
1   -121 days +03:01:00
2   -167 days +03:01:00
3    -88 days +03:01:00
dtype: timedelta64[ns]

In [29]:
sol_log.submission_time.dropna().max() - sol_log.submission_time.dropna().min()

Timedelta('627 days 11:05:02.536191')

In [45]:
#data_prefix = "ling_data/"
data_prefix = "data/"
course = pd.read_csv(data_prefix + "course.csv")
course_element = pd.read_csv(data_prefix + "course_element.csv") 
course_module = pd.read_csv(data_prefix + "course_module.csv") 
sol_log = pd.read_csv(data_prefix + "solution_log.csv") 
u_course_progress = pd.read_csv(data_prefix + "user_course_progress.csv") 
u_element_progress = pd.read_csv(data_prefix + "user_element_progress.csv") 
u_module_progress = pd.read_csv(data_prefix + "user_module_progress.csv")
users = pd.read_csv("user_course_ids.csv")

sol_log.submission_time = pd.to_datetime(sol_log.submission_time)
u_element_progress.time_updated = pd.to_datetime(u_element_progress.time_updated)
u_module_progress.time_unlocked = pd.to_datetime(u_module_progress.time_unlocked)
u_element_progress.time_achieved = pd.to_datetime(u_element_progress.time_achieved)

u_course_progress = u_course_progress.merge(users, on=['user_id', 'course_id'])

C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
## sub_time = u_element_progress.merge(sol_log, left_on='id', right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]
def calc_active_days_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log, sub_time):
    sub_time = sub_time[(sub_time.submission_time < date) & (sub_time.submission_time > date - timedelta(days=window_size))]
    sub_time.submission_time = sub_time.submission_time.dt.round("D")
    d = dict(sub_time.groupby(["user_id"]).submission_time.nunique())

    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id.unique()
    result["factor_value"] = result.user_id.map(d).fillna(0) / window_size


    
    return result


    
def calc_progress_current_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    u_element_progress.progress_current = u_element_progress.progress_current.astype(bool)
    d = u_element_progress[(u_element_progress.achieve_reason == "solved") & (u_element_progress.course_element_type == "task") &
                   (u_element_progress.time_updated < date) & (u_element_progress.time_updated > date - timedelta(days=window_size))].groupby(["user_id", "course_id"]).progress_current.sum()
    
    
    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id
    result["course_id"] = u_course_progress.course_id
    result["progress_current"] = result.merge(d, on=["user_id", "course_id"], how="left").progress_current.fillna(0)
    return result







def calc_task_completeness_points_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    module2totalpoints = course_element[course_element.element_type == "task"].groupby("module_id").score.sum()
    
    module2tasks = dict(course_element[course_element.element_type == "task"].groupby("module_id").id.count())
    u_module_progress["task_count"] = u_module_progress.course_module_id.map(module2tasks)

    uidcid2possible_tasks = u_module_progress[u_module_progress.time_unlocked < date].groupby(["user_id", "course_id"]).task_count.sum()
    uidcid2solved_before = u_element_progress[(u_element_progress.time_achieved <= date - timedelta(days=window_size)) & (u_element_progress.course_element_type == "task")].groupby(["user_id", "course_id"]).progress_current.sum()
    period_progress = calc_progress_current_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                                  u_element_progress, sol_log)


    uidcid2solved_before.name = "solved_before"
    uidcid2possible_tasks.name = "possible_tasks"
    
    period_progress = period_progress.merge(uidcid2solved_before, on=["user_id", "course_id"], how="left").fillna(0).merge(uidcid2possible_tasks, on=["user_id", "course_id"], how="left").dropna()
    result = pd.DataFrame()
    result["user_id"] = period_progress.user_id
    result["course_id"] = period_progress.course_id
    result["task_completeness"] = period_progress.progress_current / (period_progress.possible_tasks - period_progress.solved_before)
    return result






## Категориальная характеристика основанная на порядке решения
def calc_order_for_user(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    data1 = u_element_progress.loc[((u_element_progress['course_element_type'] == 'video') | (u_element_progress['course_element_type'] == 'text')) &
                              ((u_element_progress.time_updated < date) & (u_element_progress.time_updated > date - timedelta(days=window_size)))]
    data1 = data1.merge(course_element, how  = 'inner', left_on = 'course_element_id', right_on= 'element_id' )
    data1 = data1.sort_values(['user_id', 'course_id', 'course_module_id', 'time_achieved'])

    kl = pd.DataFrame(columns = ['element_progress_id', 'submission_time'], )

    kl['element_progress_id'] = sol_log['element_progress_id']
    kl['submission_time'] = sol_log['submission_time']
    kl = kl.groupby(['element_progress_id']).min('submission_time').reset_index()


    data2 = u_element_progress.merge(kl, how = 'left', left_on = 'id', right_on = 'element_progress_id')
    data2 = data2.loc[(data2['course_element_type'] == 'task')]
    data2 = data2.merge(course_element, how  = 'inner', left_on = 'course_element_id', right_on= 'element_id' )
    #data2 = data2.merge(user_course_ids, on=['user_id', 'course_id'])

    data2 = data2.sort_values(['user_id', 'course_id', 'course_module_id', 'submission_time']).reset_index()
    tasks = data2[['user_id', 'course_id', 'course_module_id', 'course_element_id', 'course_element_type']]
    tasks['time_try'] = data2['submission_time'] 
    tasks['position'] = data2['position']
    theory = data1[['user_id', 'course_id', 'course_module_id', 'course_element_id', 'course_element_type']]
    theory['time_try'] = data1['time_achieved'] 
    theory['position'] = data1['position']
    frames = [tasks, theory]
    
    
    distribution = pd.concat(frames)
    distribution = distribution.sort_values(['user_id', 'course_id', 'course_module_id', 'time_try']).reset_index()
    distribution = distribution.drop(['index'], axis = 1).drop_duplicates().reset_index()
    distribution1 = distribution.sort_values(['user_id', 'course_id', 'course_module_id', 'position']).reset_index()

    distribution1['order'] = (distribution1['position'] == distribution['position'])
    distribution1['order'] = ~distribution1['order']
    new_obj = pd.DataFrame(distribution1.groupby(['user_id', 'course_id']).order.sum())
    new_obj[new_obj["order"] > 0] = 1
    new_obj.rename(columns={"order" : "factor_value"})
    return new_obj





In [86]:
calc_order_for_user(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log)

<ipython-input-85-7848b98a4f38>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tasks['time_try'] = data2['submission_time']
<ipython-input-85-7848b98a4f38>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tasks['position'] = data2['position']
<ipython-input-85-7848b98a4f38>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,,order
user_id,course_id,
1034,3,0
3971,3,0
100119700110000264,3,0
100119700110000266,3,0
100119700110000274,3,0
...,...,...
100120190110570532,3,0
100120190110586064,3,0
100120190110592083,3,0


In [37]:
sub_time = u_element_progress.merge(sol_log, left_on='id', right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]

In [26]:
calc_current_score_for_users(date, 0, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log).sum()

user_id        -1.997708e+18
factor_value    0.000000e+00
dtype: float64

In [16]:



## Поинты за период / размер периода
def calc_period_mean_speed_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log, sub_time):
    progress = calc_progress_current_points_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log)
    result = pd.DataFrame()
    result["user_id"] = progress.user_id
    result = result.merge(progress, on="user_id", how="left").fillna(0)
    print(result.columns)
    result["factor_value"] = result.progress_current / window_size
    result.drop(columns=["progress_current"], inplace=True)
    return result

## Поинты за период / активные дни 
def calc_active_days_mean_speed_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log, sub_time):
    active_days = calc_active_days_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log, sub_time)
    progress = calc_progress_current_points_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log)
    result = pd.DataFrame()
    result["user_id"] = progress.user_id
    result = result.merge(progress, on="user_id", how="left").merge(active_days, on="user_id", how="left").fillna(0)
    result["factor_value"] = result.progress_current / result.factor_value
    result.factor_value = result.factor_value.fillna(0)
    result.drop(columns=["progress_current"], inplace=True)
    return result


## user progress in (date - window_size days, date)
def calc_progress_current_points_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    d = u_element_progress[(u_element_progress.achieve_reason == "solved") & (u_element_progress.course_element_type == "task") &
                   (u_element_progress.time_updated < date) & (u_element_progress.time_updated > date - timedelta(days=window_size))].groupby(["user_id", "course_id"]).progress_current.sum()
    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id
    result["course_id"] = u_course_progress.course_id
    result["progress_current"] = result.merge(d, on=["user_id", "course_id"], how="left").progress_current.fillna(0)
    result.progress_current = result.user_id.map(dict(result.groupby("user_id").progress_current.mean()))
    result.drop(columns=["course_id"], inplace=True)
    return result


## Доля полученных баллов из возможных за период
def calc_current_score_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    module2totalpoints = course_element[course_element.element_type == "task"].groupby("module_id").score.count()
    module2tasks = dict(course_element[course_element.element_type == "task"].groupby("module_id").id.count())
    u_module_progress["task_count"] = u_module_progress.course_module_id.map(module2tasks).fillna(0)

    
    uidcid2possible_tasks = u_module_progress[u_module_progress.time_unlocked < date].groupby(["user_id"]).task_count.sum()
    uidcid2solved_before = u_element_progress[(u_element_progress.time_achieved <= date - timedelta(days=window_size)) &
                                              (u_element_progress.course_element_type == "task")].groupby(["user_id"]).progress_current.sum()
    period_progress = calc_progress_current_points_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                                  u_element_progress, sol_log)


    uidcid2solved_before.name = "solved_before"
    uidcid2possible_tasks.name = "possible_tasks"
    
    period_progress = period_progress.merge(uidcid2solved_before, on=["user_id"], how="left").fillna(0).merge(uidcid2possible_tasks, on=["user_id"], how="left").dropna()
    result = pd.DataFrame()
    result["user_id"] = period_progress.user_id
    result["factor_value"] = period_progress.progress_current / (period_progress.possible_tasks - period_progress.solved_before)
    
    return result


def calc_progress_current_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    u_element_progress.progress_current = u_element_progress.progress_current.astype(bool)
    d = u_element_progress[(u_element_progress.achieve_reason == "solved") & (u_element_progress.course_element_type == "task") &
                   (u_element_progress.time_updated < date) & (u_element_progress.time_updated > date - timedelta(days=window_size))].groupby(["user_id", "course_id"]).progress_current.sum()
    
    
    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id
    result["course_id"] = u_course_progress.course_id
    result["progress_current"] = result.merge(d, on=["user_id", "course_id"], how="left").progress_current.fillna(0)
    return result




def calc_progress_current_for_users_text(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    d = u_element_progress[(u_element_progress.achieve_reason == "solved") & (u_element_progress.course_element_type.isin(["text", "video"])) &
                   (u_element_progress.time_updated < date) & (u_element_progress.time_updated > date - timedelta(days=window_size))].groupby(["user_id", "course_id"]).progress_current.count()
    
    
    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id
    result["course_id"] = u_course_progress.course_id
    result["progress_current"] = result.merge(d, on=["user_id", "course_id"], how="left").progress_current.fillna(0)
    return result


## "доля" выполненных заданий
def calc_task_completeness_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    module2totalpoints = course_element[course_element.element_type == "task"].groupby("module_id").score.count()
    
    module2tasks = dict(course_element[course_element.element_type == "task"].groupby("module_id").id.count())
    u_module_progress["task_count"] = u_module_progress.course_module_id.map(module2tasks).fillna(0)

    uidcid2possible_tasks = u_module_progress[u_module_progress.time_unlocked < date].groupby(["user_id"]).task_count.sum()
    uidcid2solved_before = u_element_progress[(u_element_progress.time_achieved <= date - timedelta(days=window_size)) &
                                              (u_element_progress.course_element_type == "task")].groupby(["user_id"]).progress_current.sum()
    period_progress = calc_progress_current_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                                  u_element_progress, sol_log)


    uidcid2solved_before.name = "solved_before"
    uidcid2possible_tasks.name = "possible_tasks"
    
    period_progress = period_progress.merge(uidcid2solved_before, on=["user_id"], how="left").fillna(0).merge(uidcid2possible_tasks, on=["user_id"], how="left").dropna()
    result = pd.DataFrame()
    result["user_id"] = period_progress.user_id
    result["course_id"] = period_progress.course_id
    result["task_completeness"] = period_progress.progress_current / (period_progress.possible_tasks - period_progress.solved_before)
    d = dict(result.groupby(["user_id"]).task_completeness.mean())
    result.drop(columns=["course_id", "task_completeness"], inplace=True)
    result["factor_value"] = result.user_id.map(d).fillna(0)
    
    return result


## "доля" выполненных текстов и видео
def calc_text_completeness_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    module2totalpoints = course_element[course_element.element_type.isin(["text", "video"])].groupby("module_id").score.count()
    
    module2tasks = dict(course_element[course_element.element_type.isin(["text", "video"])].groupby("module_id").id.count())
    u_module_progress["task_count"] = u_module_progress.course_module_id.map(module2tasks).fillna(0)

    uidcid2possible_tasks = u_module_progress[u_module_progress.time_unlocked < date].groupby(["user_id"]).task_count.sum()
    uidcid2solved_before = u_element_progress[(u_element_progress.time_achieved <= date - timedelta(days=window_size)) &
                                              (u_element_progress.course_element_type.isin(["text", "video"]))].groupby(["user_id"]).progress_current.sum()
    period_progress = calc_progress_current_for_users_text(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                                  u_element_progress, sol_log)


    uidcid2solved_before.name = "solved_before"
    uidcid2possible_tasks.name = "possible_tasks"
    
    period_progress = period_progress.merge(uidcid2solved_before, on=["user_id"], how="left").fillna(0).merge(uidcid2possible_tasks, on=["user_id"], how="left").dropna()
    result = pd.DataFrame()
    result["user_id"] = period_progress.user_id
    result["course_id"] = period_progress.course_id
    result["task_completeness"] = period_progress.progress_current / (period_progress.possible_tasks - period_progress.solved_before)
    d = dict(result.groupby(["user_id"]).task_completeness.mean())
    result.drop(columns=["course_id", "task_completeness"], inplace=True)
    result["factor_value"] = result.user_id.map(d).fillna(0)
    
    return result



## подсчет попыток (в абсолютных значениях)
def calc_tries_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id.unique()
    result["factor_value"] = result.user_id.map(dict(u_element_progress[(u_element_progress.time_updated < date) & (u_element_progress.time_updated > date - timedelta(days=window_size))].groupby("user_id").tries_count.sum())).fillna(0)
    return result

1.0

In [295]:
ans1 = calc_active_days_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log, sub_time)

C:\Users\Sirius\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Фичи: 
course_done - получен ли сертификат 1 уровня
progress_current_ratio - доля полученных баллов за курс
progress_current - сколько всего баллов полученно 
total_courses_done - сколько всего закрыл курсов ученик
total_tries_overall - сколько попыток сдачи сделал на всех курсах 
tries_count - сколько попыток сделал ученик на данном курсе
task_problems_in_course	text_problems_in_course	video_problems_in_course - сколько в курсе соответсвующих задач
mean_progress - на сколько баллов люди в среднем прорешивают курс
total_participants - сколько всего участников было на курсе
advanced_modules_count - сколько в курсе продвинутых модулей (в геометрии 0)
transferred_count - сколько задач пользователем перенесено
tries_efficiency - кол-во баллов в курсе делить на кол-во попыток в курсе
active_days - кол-во активных дней работы

То что еще надо юзать: 
knowledge_area_id

In [119]:
## Посчитаем, сколько в курсе можно заработать максимум баллов
course_to_max_points = dict(course_module.groupby("course_id").progress_max.sum())


## Подсчитаем, закрыл ли пользователь курс на сертификат первого уровня

d = dict(course_module[~course_module.is_advanced & course_module.level == 1].groupby(by="course_id").id.unique().apply(len))
u_course_progress["course_done"] = u_course_progress.course_id.map(d) <= u_course_progress.modules_achieved


## Подсчитаем по пользователю, сколько баллов он заработал

u_course_progress = u_course_progress.merge(u_element_progress.groupby(['user_id', 'course_id']).\
                                            progress_current.sum(), on=['user_id', 'course_id'])

## Найдем долю баллов которую получил пользователь от максимального числа баллов

u_course_progress["progress_current_ratio"] = u_course_progress.progress_current / u_course_progress.course_id.map(course_to_max_points)


## Найдем, сколько всего ученик закрыл курсов

u_course_progress["total_courses_done"] = u_course_progress.user_id.map(dict(u_course_progress.groupby("user_id").course_done.sum()))





## Найдем, сколько всего попыток сдачи задач, сделал ученик. total_tries_overall - считая все курсы, 
## total_tries - учитывая только данный курс

u_course_progress["total_tries_overall"] = u_course_progress.user_id.map(dict(u_element_progress.groupby("user_id").tries_count.sum()))
u_course_progress = u_course_progress.merge( u_element_progress.groupby(["user_id", "course_id"]) \
                        .tries_count.sum(), on=['user_id', 'course_id'], how = "left")





## Найдем, сколько в курсе видео / текстовых / задачный элементов

course_element["course_id"] = course_element.module_id.map(dict(zip(course_module.id, course_module.course_id)))

for name in ["task", "text", "video"]:
    u_course_progress[name + "_problems_in_course"] = u_course_progress.course_id. \
    map(dict(course_element[course_element.element_type == name].groupby("course_id").id.count()))
    
elem_count = u_course_progress["task_problems_in_course"] + u_course_progress["video_problems_in_course"] + u_course_progress["text_problems_in_course"]

for name in ["task", "text", "video"]:
    u_course_progress[name + "_problems_in_course"] /= elem_count
     


## Найдем, сколько баллов получают люди за курс в среднем

u_course_progress["mean_progress"] = u_course_progress.course_id.map(dict(u_course_progress.groupby("course_id").progress_current_ratio.sum() / u_course_progress.groupby("course_id").progress_current_ratio.count()))


## Найдем, сколько всего участников на курсе


u_course_progress["total_participants"] = u_course_progress.course_id.map(dict(u_course_progress.groupby("course_id").id.count()))



## Найдем, сколько продвинутых модулей на курсе
u_course_progress["advanced_modules_count"] = u_course_progress.course_id.map(course_module.groupby("course_id").is_advanced.sum())



## Посчитаем, сколько задач каждый пользователь перенес 

d = u_element_progress[u_element_progress.achieve_reason == "transferred"].groupby(["user_id", "course_id"]).id.count()
u_course_progress["transferred_count"] = u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").id_y.fillna(0)


## Посчитаем кол-во баллов полученное за курс учеников разделенное на количество попыток в курсе
u_course_progress["tries_efficiency"] = u_course_progress.progress_current / u_course_progress.tries_count


## Посчитаем  дни активности

sub_time = u_element_progress.merge(sol_log, left_on='id', right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]

sub_time.submission_time = pd.to_datetime(sub_time.submission_time)
sub_time.submission_time = sub_time.submission_time.dt.round("D")

d = sub_time.groupby(["user_id", "course_id"]).submission_time.nunique()

u_course_progress["active_days"] = u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").submission_time.fillna(0)


## Найдем баллы деленные на кол-во дней активности (скорость набора баллов)
u_course_progress["solving_speed"] = u_course_progress.progress_current / u_course_progress.active_days


## я не знаю что это и где оно появляется
u_course_progress.drop(columns='Unnamed: 0', inplace=True)


u_course_progress.to_csv("to_combine/user_course_progress_done.csv", index=None)


,id,user_id,course_id,course_module_id,course_element_type,course_element_id,progress_current,is_achieved,hints_used,tries_count,module_progress_id,time_created,time_updated,time_closed,time_achieved,time_started,achieve_reason
5,42748471,100120080111073572,89,1199,task,1861,1.0,True,0,1.0,2246219,2020-04-27 13:01:03.723581,2020-07-28 12:32:20.397575,2020-07-28 12:32:20.397575,2020-07-28 12:32:20.397575,NaN,solved
6,42748470,100120080111073572,89,1199,task,1860,1.0,True,0,1.0,2246219,2020-04-27 13:01:03.723581,2020-07-28 12:22:52.545546,2020-07-28 12:22:52.545546,2020-07-28 12:22:52.545546,NaN,solved
7,42748469,100120080111073572,89,1199,task,1859,1.0,True,0,2.0,2246219,2020-04-27 13:01:03.723581,2020-07-28 12:18:53.789370,2020-07-28 12:18:53.789370,2020-07-28 12:18:53.789370,NaN,solved
8,42748468,100120080111073572,89,1199,task,860,1.0,True,0,3.0,2246219,2020-04-27 13:01:03.723581,2020-07-28 12:08:36.679552,2020-07-28 12:08:36.679552,2020-07-28 12:08:36.679552,NaN,solved
9,42748467,100120080111073572,89,1199,task,1858,1.0,True,0,1.0,2246219,2020-04-27 13:01:03.723581,2020-07-28 12:05:34.961752,2020-07-28 12:05:34.961752,2020-07-28 12:05:34.961752,NaN,solved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10108589,50342007,100120040111407406,89,1213,video,73,0.0,True,0,0.0,2635829,2020-05-25 17:13:24.511002,2020-07-29 19:59:09.988520,2020-07-29 19:59:09.988520,2020-07-29 19:59:09.988520,NaN,solved
10150928,52179168,100120050111330401,89,1190,task,2122,0.0,NaN,0,15.0,2736515,2020-06-06 10:25:43.610094,2020-07-18 08:26:25.014577,NaN,NaT,NaN,NaN
10181958,50963688,100120040110747801,89,1196,video,355,0.0,True,0,0.0,2670699,2020-05-29 13:47:49.338147,2020-07-22 21:29:16.276982,2020-07-22 21:29:16.276982,2020-07-22 21:29:16.276982,NaN,solved
10188496,50341667,100120040111407406,89,1197,video,354,0.0,True,0,0.0,2635809,2020-05-25 17:13:24.511002,2020-07-29 19:58:33.979000,2020-07-29 19:58:33.979000,2020-07-29 19:58:33.979000,NaN,solved


In [104]:
result

,user_id,factor_value
0,100120050110043028,0.0
1,100120060110982009,0.0
2,100120050110554181,0.0
3,100120040110288242,0.0
4,100120040110523247,0.0
...,...,...
14381,100120060110754725,0.0
14382,100120040111601218,0.0
14383,100120050111090974,0.0
14384,100120070111186046,0.0


In [101]:
u_course_progress

,id,user_id,course_id,modules_achieved,advanced_modules_done,time_created,time_updated,modules_closed,modules_done,Unnamed: 0,total_tries_overall
0,43519,100120050110043028,3,0,0,2019-05-06 19:22:12.861890,2019-06-05 16:45:32.140810,0,0,5532,0.0
1,2681,100120060110982009,3,8,2,2019-03-22 15:42:21.255839,2019-06-14 11:48:47.039643,1,2,10478,0.0
2,29960,100120050110554181,3,2,1,2019-04-30 15:23:41.502043,2019-06-05 16:45:32.140810,2,2,7585,0.0
3,27474,100120040110288242,3,5,1,2019-04-18 18:48:44.483242,2019-06-15 18:39:16.862691,4,4,4381,0.0
4,30161,100120040110523247,3,2,0,2019-05-01 06:34:08.301213,2019-06-05 16:45:32.140810,1,1,4763,0.0
...,...,...,...,...,...,...,...,...,...,...,...
16598,212873,100120050111240277,237,4,0,2020-09-29 08:19:47.467425,2020-09-29 08:20:08.928585,3,3,8443,0.0
16599,211756,100120040111038923,237,3,0,2020-09-27 07:13:29.772199,2020-09-27 07:13:40.104542,3,3,4987,0.0
16600,209654,100120050111090974,237,1,0,2020-09-23 20:50:51.737021,2020-09-23 21:16:08.026812,1,1,8225,0.0
16601,208123,100120070111186046,237,2,0,2020-09-22 08:12:23.649605,2020-09-22 19:48:47.303804,1,2,15192,0.0


In [8]:
from datetime import timedelta
import datetime

In [9]:
window_size = 14

In [10]:
date = pd.to_datetime('2020-07-30') 

In [4]:
sub_time = u_element_progress.merge(sol_log, left_on='id', right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]

sub_time.submission_time = pd.to_datetime(sub_time.submission_time)
sub_time.submission_time = sub_time.submission_time.dt.round("D")


d = sub_time.groupby(["user_id", "course_id"]).submission_time.nunique()

u_course_progress["active_days"] = u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").submission_time.fillna(0)